<a href="https://colab.research.google.com/github/STLNFTART/MotorHandPro/blob/main/notebooks/experiments/01_parameter_sweep_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameter Sweep Analysis

Systematic exploration of Primal Logic parameters:
- KE (exponential gain) sweeps
- Lambda (decay rate) variations
- Sensitivity analysis
- Heatmap visualizations

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install numpy matplotlib pandas seaborn
    !git clone https://github.com/STLNFTART/MotorHandPro.git
    sys.path.append('/content/MotorHandPro')
    %cd /content/MotorHandPro
else:
    sys.path.append('../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pathlib import Path

## 1. Load Experiment Results

Load existing experiment data from the `experiments/runs/` directory.

In [ ]:
# Find all experiment CSV files
experiments_path = Path('experiments/runs') if Path('experiments/runs').exists() else Path('.')
csv_files = list(experiments_path.glob('*.csv'))

print(f"Found {len(csv_files)} experiment files")
for f in csv_files[:5]:
    print(f"  - {f.name}")
if len(csv_files) > 5:
    print(f"  ... and {len(csv_files) - 5} more")

## 2. Parameter Sweep Simulation

Run a parameter sweep if no data is available.

In [ ]:
class PrimalLogic:
    def __init__(self, KE=0.5, lambda_val=0.16905):
        self.KE = KE
        self.lambda_val = lambda_val
        self.Ec = 0.0
        
    def update(self, error, dt):
        decay = np.exp(-self.lambda_val * dt)
        self.Ec = self.Ec * decay + error * dt
        psi = error + self.lambda_val * self.Ec
        gamma = 149.9992314 * (1.0 - np.exp(-abs(psi) / 149.9992314))
        return psi, gamma, self.Ec

def run_sweep(KE_values, lambda_values, duration=10.0, dt=0.01):
    """Run parameter sweep"""
    results = []
    t = np.arange(0, duration, dt)
    
    for KE in KE_values:
        for lambda_val in lambda_values:
            controller = PrimalLogic(KE=KE, lambda_val=lambda_val)
            state = 0.0
            setpoint = 1.0
            
            settling_time = None
            overshoot = 0.0
            
            for i, ti in enumerate(t):
                error = setpoint - state
                psi, gamma, Ec = controller.update(error, dt)
                state += dt * (-state + psi)
                
                # Track metrics
                if state > overshoot:
                    overshoot = state
                if settling_time is None and abs(state - setpoint) < 0.02:
                    settling_time = ti
            
            final_error = abs(state - setpoint)
            overshoot_pct = (overshoot - setpoint) / setpoint * 100
            
            results.append({
                'KE': KE,
                'lambda': lambda_val,
                'settling_time': settling_time if settling_time else duration,
                'overshoot_pct': overshoot_pct,
                'final_error': final_error
            })
    
    return pd.DataFrame(results)

# Run sweep
KE_vals = np.linspace(0.1, 2.0, 15)
lambda_vals = np.linspace(0.05, 0.5, 15)

print("Running parameter sweep...")
df_sweep = run_sweep(KE_vals, lambda_vals)
print(f"Completed {len(df_sweep)} parameter combinations")
df_sweep.head()

## 3. Heatmap Visualizations

In [ ]:
# Create heatmaps
metrics = ['settling_time', 'overshoot_pct', 'final_error']
titles = ['Settling Time (s)', 'Overshoot (%)', 'Final Error']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, metric, title in zip(axes, metrics, titles):
    pivot = df_sweep.pivot(index='lambda', columns='KE', values=metric)
    sns.heatmap(pivot, ax=ax, cmap='RdYlGn_r', annot=False, fmt='.2f', cbar_kws={'label': title})
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('KE', fontsize=12)
    ax.set_ylabel('Lambda', fontsize=12)

plt.tight_layout()
plt.show()

## 4. Optimal Parameter Selection

In [ ]:
# Find optimal parameters
optimal_idx = df_sweep['final_error'].idxmin()
optimal_params = df_sweep.loc[optimal_idx]

print("\nOptimal Parameters:")
print("="*50)
print(f"KE: {optimal_params['KE']:.4f}")
print(f"Lambda: {optimal_params['lambda']:.4f}")
print(f"Settling time: {optimal_params['settling_time']:.3f}s")
print(f"Overshoot: {optimal_params['overshoot_pct']:.2f}%")
print(f"Final error: {optimal_params['final_error']:.6f}")

## Summary

This notebook demonstrates systematic parameter exploration for the Primal Logic controller. Key insights:

1. **KE** controls memory influence
2. **Lambda** controls decay rate
3. Trade-offs between speed and stability
4. Optimal regions in parameter space